In [1]:
import geopandas as gpd
import pandas as pd
import pickle
import momepy
import dask
from dask.distributed import Client, LocalCluster
from tqdm.auto import tqdm
from scripts.percentile import Percentiles


/tmp/ipykernel_8000/1586022436.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
daskCluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(daskCluster, timeout="300s")

client



2023-04-25 00:35:40,359 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-25 00:35:40,361 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-25 00:35:40,364 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-25 00:35:40,366 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-25 00:35:40,368 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-25 00:35:40,371 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-25 00:35:40,374 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-25 00:35:40,

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 250.47 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35527,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 250.47 GiB
Comm: tcp://127.0.0.1:33827,Total threads: 2
Dashboard: http://127.0.0.1:42085/status,Memory: 31.31 GiB
Nanny: tcp://127.0.0.1:33371,


In [3]:
tessellation = gpd.read_parquet("./out/singapore/tessellation_stats.pq")
buildings = gpd.read_parquet("./out/singapore/buildings.pq")

with open('./out/singapore/weights.pickle', 'rb') as f:
    queen_1 = pickle.load(f)
    queen_3 = pickle.load(f)
    dist200 = pickle.load(f)
    f.close()
    

In [4]:
percentiles = []
for column in tqdm(tessellation.columns.drop(["geometry", "uID"]), total = len(tessellation.columns.drop(["geometry", "uID"]))):
    perc = Percentiles(tessellation, column, queen_3, "uID", verbose=True, client=client).frame
    perc.columns = [f"{column}_" + str(x) for x in perc.columns]
    percentiles.append(perc)

  0%|          | 0/33 [00:00<?, ?it/s]

<string>:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
/home/virgilxw/miniconda3/lib/python3.10/site-packages/distributed/client.py:3095: UserWarning: Sending large graph of size 11.16 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
<string>:1: UserWarning: Shapely 2.0 is installed, bu

In [ ]:
percentiles_joined = pd.concat(percentiles, axis=1)
percentiles_joined.head()